https://www.kaggle.com/misrakahmed/vegetable-image-dataset

데이터를 분류하는 데에 여러 모델들을 사용해 보고 성능, 학습 시간 비교

In [2]:
# 필수 코드

import torch
import torchvision

device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.random.manual_seed(0)
if device == 'cuda':
    torch.cuda.manual_seed_all(0)